# Install PuLP via pip

In [1]:
!pip install pulp


# Import and Global Variables

In [2]:
from pulp import *


# Setup Constants Function

In [3]:
# TODO: Redefine for n instead of just for 9
def define_constants(n):
  Vals = range(1, n+1)  # Sequence of 1 to n
  Rows = range(0, n)    # Sequence of 0 to n-1 (Since Python is 0-based)
  Cols = range(0, n)    # Sequence of 0 to n-1 (Since Python is 0-based)

  if n % 3 != 0:
    raise Exception('%s needs to be divisible by 3')

  SubN = int(n/3)       
  Sectors = []

  for i in range(SubN):
      for j in range(SubN):
          Sectors += [
              [
                (
                    Rows[SubN*i+k],Cols[SubN*j+l]
                ) for k in range(SubN) for l in range(SubN)
              ]
            ]

  return Vals, Rows, Cols, Sectors

Vals, Rows, Cols, Sectors = define_constants(9)

# Define Standard Constraints
* 1 Value per Option
* Single Value per Row, Column, and Sector

In [4]:
def define_problem_and_constraints(Vals, Rows, Cols):
  problem = LpProblem("Sudoku", LpMinimize)
  # Objective Function is irrelevant as we are just looking for an optimal soln
  problem += 0

  options = LpVariable.dicts("Options", (Vals,Rows,Cols), 0, 1, LpInteger)
  
  # Constraint: Single Value per Option
  for r in Rows:
    for c in Cols:
        problem += lpSum(
            [options[v][r][c] for v in Vals]
        ) == 1

  # Constraint: Single Value per Row, Column, and Sector
  for v in Vals:
    for r in Rows:
        problem += lpSum(
            [options[v][r][c] for c in Cols]
        ) == 1
        
    for c in Cols:
        problem += lpSum(
            [options[v][r][c] for r in Rows]
        ) == 1

    for b in Sectors:
        problem += lpSum(
            [options[v][r][c] for (r,c) in b]
        ) == 1

  return problem, options

problem, options = define_problem_and_constraints(Vals, Rows, Cols)

# Define Sudoku Problem and Solve

In [6]:
def define_values_and_solve(problem, options, values):
  for r_idx, row in enumerate(values):
    for c_idx, value in enumerate(row):
      if value > 0:
        problem += options[value][r_idx][c_idx] == 1
  
  problem.solve()

  if LpStatus[problem.status] == 'Optimal':
    print('Solution Found. Problem is optimal and feasible.')
  else:
    print('Infeasible Problem.')

  return problem

# 3-D Array where 0 is not a constraint
# TODO: Take in CSV and output/compare solution
sudoku_problem = [
  [5,3,0,0,7,0,0,0,0],
  [6,0,0,1,9,5,0,0,0],
  [0,9,8,0,0,0,0,6,0],
  [8,0,0,0,6,0,0,0,3],
  [4,0,0,8,0,3,0,0,1],
  [7,0,0,0,2,0,0,0,6],
  [0,6,0,0,0,0,2,8,0],
  [0,0,0,4,1,9,0,0,5],
  [0,0,0,0,8,0,0,7,9]
]

problem = define_values_and_solve(problem, options, sudoku_problem)

Solution Found. Problem is optimal and feasible.


# Grab Optimal Values and Conduct Python Array

In [37]:
def construct_solution_matrix(Rows, Cols, Vals, options):
  soln = []
  n = len(Rows)

  for r in Rows:
      row = []

      for c in Cols:
          for v in Vals:
            if value(options[v][r][c]):
              row.append(v)
              
              if c == n-1:
                soln.append(row)
                row = []
  return soln

solution = construct_solution_matrix(Rows, Cols, Vals, options)
print('Solution = ')
for s in solution:
  print(s)

Solution = 
[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]


# Construct User-Readable Print-out

In [36]:
def pretty_print_solution(soln):
  n = len(soln)
  sub_n = len(soln)/3

  line = 3*n*'-' + '\n'
  prnt = ''

  for r_idx, row in enumerate(soln):
    if r_idx % sub_n == 0:
      prnt += line

    for c_idx, value in enumerate(row):
      if c_idx % sub_n == 0:
        prnt += ' |'

      prnt += ' ' + str(value)
    prnt += ' |'


    prnt += '\n'
  prnt += line
  print(prnt)

pretty_print_solution(solution)

---------------------------
 | 5 3 4 | 6 7 8 | 9 1 2 |
 | 6 7 2 | 1 9 5 | 3 4 8 |
 | 1 9 8 | 3 4 2 | 5 6 7 |
---------------------------
 | 8 5 9 | 7 6 1 | 4 2 3 |
 | 4 2 6 | 8 5 3 | 7 9 1 |
 | 7 1 3 | 9 2 4 | 8 5 6 |
---------------------------
 | 9 6 1 | 5 3 7 | 2 8 4 |
 | 2 8 7 | 4 1 9 | 6 3 5 |
 | 3 4 5 | 2 8 6 | 1 7 9 |
---------------------------

